In [1]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()  

geemap package not installed. Installing ...
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=pfVIwvQULbprP6-LpU-BU3XF_U-SjS7nvLbUylwG9RM&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWioL7A-6DraPnWylzMag36frkvhvGeppP5kZr7AQZhhWTecROhBxVA

Successfully saved authorization token.


In [2]:
#Connect to Google Drive if you want to export images
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Go to correct folder in Google Drive
%cd /content/drive/MyDrive/spatial_colab/
%ls

/content/drive/MyDrive/spatial_colab
datasets/  pod_pou_lulcc/


In [ ]:
# Algorithm:
# 1. Get huc value for watershed of interest (see table below)
# 3. Create a new feature with selected watershed
# 5. Union the watersheds (if necessary), export the watershed(s)
# 6. Get the CDL data
# 7. Clip to the area of the watershed
# 8. Export 

#--------------------------------#
#   Name           |    huc8     #
#--------------------------------#
#   Lower Boise    |  17050114   #
#--------------------------------#

In [4]:
wshd = ee.FeatureCollection("USGS/WBD/2017/HUC08")

lbrb_wshd = wshd.filterMetadata('name', 'equals', 'Lower Boise')

#print to map
map = emap.Map()
map.addLayer(ee.Image().paint(lbrb_wshd, 0, 2), {}, 'Lower Boise River Basin')
map.addLayerControl()
map

In [11]:
dataset = ee.ImageCollection("UMT/Climate/IrrMapper_RF/v1_0")
lbrb_irrmapper = dataset.map(lambda image: image.clip(lbrb_wshd))

irrigatedlandVis = {
    'min': 0.0,
    'max': 1.0,
    'palette': ['blue']
}

Map = emap.Map(center=(43.6150, -116.2023),zoom=8)
Map.addLayer(lbrb_irrmapper, irrigatedlandVis)
Map

In [12]:
downConfig = {'scale': 30, "maxPixels": 1.0E13, 'driveFolder': 'spatial_colab', 'fileFormat': 'GeoTIFF'}  # scale means resolution.
img_lst = lbrb_irrmapper.toList(100)

count = lbrb_irrmapper.size().getInfo()

for i in range(0, count):
    image = ee.Image(img_lst.get(i))
    name = image.get('system:index').getInfo()
    print(name)
    task = ee.batch.Export.image(image.clip(lbrb_wshd.geometry()), name, downConfig)
    task.start()
    print(task.status())

1986
{'state': 'READY', 'description': '1986', 'creation_timestamp_ms': 1632432614375, 'update_timestamp_ms': 1632432614375, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'D4KSCZ64Y4DFLY3LKD76SMSL', 'name': 'projects/earthengine-legacy/operations/D4KSCZ64Y4DFLY3LKD76SMSL'}
1987
{'state': 'READY', 'description': '1987', 'creation_timestamp_ms': 1632432615166, 'update_timestamp_ms': 1632432615166, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': '3NCG5SKJMN733V4TSQO7IWY2', 'name': 'projects/earthengine-legacy/operations/3NCG5SKJMN733V4TSQO7IWY2'}
1988
{'state': 'READY', 'description': '1988', 'creation_timestamp_ms': 1632432616085, 'update_timestamp_ms': 1632432616085, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'HG4HZ33E3UWCFTGAM5OPL3IQ', 'name': 'projects/earthengine-legacy/operations/HG4HZ33E3UWCFTGAM5OPL3IQ'}
1989
{'state': 'READY', 'description': '1989', 'creation_timestamp_ms': 1632432616925, 'update_timestamp_ms': 1632432616925, 'start_time

In [29]:
y2000 = lbrb_irrmapper.filterDate('2000')
y1986 = lbrb_irrmapper.filterDate('1986')
y2020 = lbrb_irrmapper.filterDate('2020')

list_irrmapper = lbrb_irrmapper.toList(lbrb_irrmapper.size())
y86 = ee.Image(list_irrmapper.get(0)).select('classification').clip(lbrb_wshd)
y20 = ee.Image(list_irrmapper.get(33)).select('classification').clip(lbrb_wshd)

#see the ratio of differences in pixels
ratio = y86.divide(y20)

#create a palette for the difference
ratioVis = {
    'min': 0.0,
    'max': 1.0,
    'palette': ['yellow', 'blue']
}
#display the ratio 
Map = emap.Map(center=(43.6150, -116.2023),zoom=8)
Map.addLayer(ratio, ratioVis)
Map

